In [1]:
# import webdriver
import urllib
from IPython.display import Image, display
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
# email sending
import smtplib
from email.mime.multipart import MIMEMultipart
from email.utils import make_msgid
import email.mime.application
from email.mime.text import MIMEText
import mimetypes
import pandas as pd
import csv
import math

# scrape data in lists- myGSOC()
# store data from lists to CSVs locally- storeScrapeData(orgs, tools)
# read csv, inputs
# search language in dictionary return orginisation name, no repetitions use set to store
# csv from set
# email

def storeScrapeData(orgs, tools, url):
    
    #orgs = np.array(orgs)
    #tools = np.array(tools)
    pd.DataFrame(orgs).to_csv("orgs.csv", header=None, index=None)
    pd.DataFrame(tools).to_csv("tools.csv", header=None, index=None)
    pd.DataFrame(url).to_csv("url.csv", header=None, index=None)

def myGSOC():
    # browser options
    options = Options()
    options.add_argument('--headless')
    # create webdriver object
    driver = webdriver.Chrome(options=options)
    URL = "https://summerofcode.withgoogle.com/archive/2020/organizations/"
    try:
        driver.get(URL)
    except:
        print("Invalid URL. Try again :(")
        return
    # class name for all organisations is organization-card__name font-black-54
    # the xpath for each differs by only 1 division ranging from 1-199, meaning total of 199 organisations
    orgs = []
    tools = [[]]
    url = []
    elements = driver.find_elements_by_class_name("organization-card__name")
    for i in range(len(elements)):
        orgs.append(elements[i].text)
        elements[i].click() # click on organisation
        url.append(driver.current_url)
        thisTools = driver.find_elements_by_class_name("organization__tag--technology") #find all technologies needed
        for j in range(len(thisTools)):
            tools[i].append(thisTools[j].text) # store these j elements in index i
        tools.append([]) # created new index i in the list
        driver.back()
        # since data is lost
        elements = driver.find_elements_by_class_name("organization-card__name")
    # because 1 extra got added in the last iteration
    tools.pop()
    storeScrapeData(orgs, tools, url)
    # now that data is stored locally we not not scrape again and again


emailId = input("Enter your email-id: ")
password = input("\nEnter password: ")
# tech stack available
print("\nRunning the Web Scraper...")

# run the next liine only 1 time
#myGSOC()

orgs = pd.read_csv('orgs.csv', header=None)
tools = pd.read_csv('tools.csv', header=None)
url = pd.read_csv('url.csv', header=None)
availableTools = set()
for i in range(len(tools)):
    for j in range(len(tools.iloc[i])):
        if not (pd.isnull(tools.iloc[i][j])):
            availableTools.add(tools.iloc[i][j])
print("\nAvailable Tech Stack: ")
for t in availableTools:
    print(t, end=", ")
print("\n")
myTools = []
addTech = ''
while addTech != 'exit':
    addTech = input("\nEnter What you know from the above tech stack. Type 'exit' when done: ").lower()
    if addTech in availableTools:
        myTools.append(addTech)
    else:
        print('\nThis tool is not from the list shown. Please select a tool from the list that you know!')
if not myTools:
    print("\nSorry, please learn a tool. Your List is empty. Terminating...")
else:
    print("\nLooking up for Organisations based on the tools you know...")
    
    # lookup each entry from myTools in tools dataframe
    #for t in myTools:
    t= myTools[0]
    pos = list()
    # Get bool dataframe with True at positions where the given value exists
    result = tools.isin([t])
    # Get list of columns that contains the value
    seriesObj = result.any()
    columnNames = list(seriesObj[seriesObj == True].index)
    # Iterate over list of columns and fetch the rows indexes where value exists
    for col in columnNames:
        rows = list(result[col][result[col] == True].index)
        for row in rows:
            pos.append((row, col))
    
    # remove repetition in the values obtained
    index = set()
    for i in range(len(pos)):
        index.add(pos[i][0])
        #print(pos[i])select
    
    # selecting only indices that are in index array from dataframes
    orgs = orgs[orgs.index.isin(index)]
    url = url[url.index.isin(index)]
    combinedTools = tools[tools.index.isin(index)]
    # merging all columns into 1 column
    combinedTools['combined'] = combinedTools[combinedTools.columns[0:]].apply(lambda x: ','.join(x.dropna().astype(str)),axis=1)
    # concat the dataframes into 1 single dataframe
    final = pd.concat([orgs, url, combinedTools['combined']], axis=1)
    final.to_csv("rsult.csv", header=None, index=None)

Enter your email-id: aa

Enter password: aa

Running the Web Scraper...

Available Tech Stack: 
https://summerofcode.withgoogle.com/archive/2020/organizations/6385935689711616/, https://summerofcode.withgoogle.com/archive/2020/organizations/4527035860385792/, https://summerofcode.withgoogle.com/archive/2020/organizations/5820969215590400/, https://summerofcode.withgoogle.com/archive/2020/organizations/4588406077652992/, https://summerofcode.withgoogle.com/archive/2020/organizations/6444776540340224/, https://summerofcode.withgoogle.com/archive/2020/organizations/5039063742021632/, https://summerofcode.withgoogle.com/archive/2020/organizations/4595548239167488/, https://summerofcode.withgoogle.com/archive/2020/organizations/5702344097923072/, https://summerofcode.withgoogle.com/archive/2020/organizations/4524333319323648/, https://summerofcode.withgoogle.com/archive/2020/organizations/6689005560659968/, https://summerofcode.withgoogle.com/archive/2020/organizations/5398756750524416/, ht


Enter What you know from the above tech stack. Type 'exit' when done: exit

This tool is not from the list shown. Please select a tool from the list that you know!

Sorry, please learn a tool. Your List is empty. Terminating...


In [2]:
availableTools

{'https://summerofcode.withgoogle.com/archive/2020/organizations/4504833932918784/',
 'https://summerofcode.withgoogle.com/archive/2020/organizations/4511020028002304/',
 'https://summerofcode.withgoogle.com/archive/2020/organizations/4524333319323648/',
 'https://summerofcode.withgoogle.com/archive/2020/organizations/4527035860385792/',
 'https://summerofcode.withgoogle.com/archive/2020/organizations/4541796253696000/',
 'https://summerofcode.withgoogle.com/archive/2020/organizations/4543471290941440/',
 'https://summerofcode.withgoogle.com/archive/2020/organizations/4549302816342016/',
 'https://summerofcode.withgoogle.com/archive/2020/organizations/4573440532545536/',
 'https://summerofcode.withgoogle.com/archive/2020/organizations/4577601315667968/',
 'https://summerofcode.withgoogle.com/archive/2020/organizations/4588406077652992/',
 'https://summerofcode.withgoogle.com/archive/2020/organizations/4595548239167488/',
 'https://summerofcode.withgoogle.com/archive/2020/organizations/

In [3]:
tools

,0
0,https://summerofcode.withgoogle.com/archive/20...
1,https://summerofcode.withgoogle.com/archive/20...
2,https://summerofcode.withgoogle.com/archive/20...
3,https://summerofcode.withgoogle.com/archive/20...
4,https://summerofcode.withgoogle.com/archive/20...
...,...
194,https://summerofcode.withgoogle.com/archive/20...
195,https://summerofcode.withgoogle.com/archive/20...
196,https://summerofcode.withgoogle.com/archive/20...
197,https://summerofcode.withgoogle.com/archive/20...


In [5]:
tools = pd.read_csv('tools.csv', header=None)
tools

,0,1,2,3,4
0,java,android,javascript,python,react
1,python,c/c++,rust,javascript,postgresql
2,c/c++,python,NaN,NaN,NaN
3,javascript,react,coq,ocaml,compiler
4,python,vhdl,raspberry pi,sqlite,c++
...,...,...,...,...,...
194,x11,opencl,opengl,wayland,vulkan
195,c++,unicode,swig,linux,golang
196,java,xmpp,vala,webrtc,lua
197,java,javascript,html5,css,velocity
